In [1]:
import os
import cv2 as cv
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras import Sequential,layers, models, losses, optimizers, datasets, utils

In [2]:
from functools import reduce
def compose(*funcs):
    """Compose arbitrarily many functions, evaluated left to right.

    Reference: https://mathieularose.com/function-composition-in-python/
    """
    if funcs:
        return reduce(lambda f, g: lambda *a, **kw: g(f(*a, **kw)), funcs)
    else:
        raise ValueError('Composition of empty sequence not supported.')

In [3]:
def inceptionCBN(
    filters,kernel_size=3,
    activation='relu',strides=1,
    padding="valid"):
    
    conv=layers.Conv2D(
        filters, kernel_size=kernel_size,
        strides=strides,padding=padding,use_bias=False)
    bn=layers.BatchNormalization(scale=False)
    layers_list=[conv,bn]
    if activation:
        act=layers.Activation(activation=activation)
        layers_list.append(act)
    return compose(*layers_list)

def reductionBlock(x,sorted_block=None):
    x1=compose(
        inceptionCBN(448,1,padding="same"),
        inceptionCBN(384,3,padding="same")
    )(x)
    x2=inceptionCBN(384,1,padding="same")(x)
    blocks=[
            layers.Concatenate()([
                inceptionCBN(384,(1,3),padding="same")(x1),
                inceptionCBN(384,(3,1),padding="same")(x1)]),
            layers.Concatenate()([
                inceptionCBN(384,(1,3),padding="same")(x2),
                inceptionCBN(384,(3,1),padding="same")(x2)]),
            compose(*[
                layers.AveragePooling2D(pool_size=3,strides=1,padding="same"),
                inceptionCBN(192,1,padding="same")])(x),
            inceptionCBN(320,1,padding="same")(x)
        ]
    if sorted_block:
        blocks = [blocks[i] for i in sorted_block]
        
    x=layers.Concatenate()(
        blocks
    )
    return x

def inceptionV3Block(input_layer, configs_list, name=None, sorted_block = [3,1,0,2]):
    blocks=list()
    for i,configs in enumerate(configs_list):
        block=list()
        for config in configs:
            if not isinstance(config, list):
                raise ValueError("The filter list is a double list.")
            if i==2:
                avg_pool=layers.AveragePooling2D(
                    pool_size=3,strides=1,padding="same")
                block.append(avg_pool)
            inception_block=inceptionCBN(*config, padding="same")
            block.append(inception_block)
        blocks.append(block)
    # model weight 비교를 위한 정렬
    blocks = [blocks[i] for i in sorted_block]
    return layers.Concatenate(
        name=None if name is None else f"mixed_{name}_{i}"
    )([compose(*i)(input_layer) for i in blocks])

In [4]:
# configs list format [[filter, kernel_size]]
configs_list1=[[[64,1],[96,3],[96,3]], [[48,1],[64,5]], [[32,1]],[[64,1]]]
configs_list2=[[[64,1],[96,3],[96,3]], [[48,1],[64,5]], [[64,1]],[[64,1]]]

configs_list3=[[[128,1],[128,(7,1)],[128,(1,7)],[128,(7,1)],[192,(1,7)]], 
               [[128,1],[128,(1,7)],[192,(7,1)]], [[192,1]],[[192,1]]]
configs_list4=[[[160,1],[160,(7,1)],[160,(1,7)],[160,(7,1)],[192,(1,7)]], 
               [[160,1],[160,(1,7)],[192,(7,1)]], [[192,1]],[[192,1]]]
configs_list5=[[[192,1],[192,(7,1)],[192,(1,7)],[192,(7,1)],[192,(1,7)]], 
               [[192,1],[192,(1,7)],[192,(7,1)]], [[192,1]],[[192,1]]]

inp = layers.Input((299,299,3))
# head
head = compose(
    inceptionCBN(32,strides=2),
    inceptionCBN(32),
    inceptionCBN(64,padding="same"),
    layers.MaxPooling2D(pool_size=3,strides=2),
    inceptionCBN(80,1),
    inceptionCBN(192),
    layers.MaxPooling2D(pool_size=3,strides=2)
)(inp)
# inception_module_A
x = inceptionV3Block(head, configs_list1)
x = inceptionV3Block(x,configs_list2)
x = inceptionV3Block(x,configs_list2)

# # grid_reduction_1
reduction_list1=[
    inceptionCBN(384,3,strides=2),
    compose(*[
        inceptionCBN(64,1,padding="same"),
        inceptionCBN(96,3,padding="same"),
        inceptionCBN(96,3,strides=2)]
           ),
    layers.MaxPooling2D(pool_size=3,strides=2)]
x=layers.Concatenate()([layer(x) for layer in reduction_list1])

# # # inception_module_B
x = inceptionV3Block(x,configs_list3)
x = inceptionV3Block(x,configs_list4)
x = inceptionV3Block(x,configs_list4)
x_a = inceptionV3Block(x,configs_list5)

# # # grid_reduction_2
reduction_list2=[
    compose(*[
        inceptionCBN(192,1,padding="same"),
        inceptionCBN(320,3,strides=2)]
        ),
    compose(*[
        inceptionCBN(192,1,padding="same"),
        inceptionCBN(192,(1,7),padding="same"),
        inceptionCBN(192,(7,1),padding="same"),
        inceptionCBN(192,3,strides=2)]
        ),
        layers.MaxPooling2D(pool_size=3,strides=2)]
x = layers.Concatenate()([layer(x_a) for layer in reduction_list2])

# # # grid_reduction_3
x = reductionBlock(x,sorted_block=[3,1,0,2])
x = reductionBlock(x,sorted_block=[3,1,0,2])

x=layers.GlobalAveragePooling2D()(x)
out=layers.Dense(units=1000,activation="softmax")(x)
copy_model=models.Model(inp,out)

2024-04-17 09:26:01.068452: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-17 09:26:01.068701: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-17 09:26:01.074319: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-17 09:26:01.074562: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-17 09:26:01.074751: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [5]:
origin_model=tf.keras.applications.InceptionV3(
    input_shape=(299,299,3),include_top=True)

copy_model.set_weights(origin_model.get_weights())

for org, new in zip(origin_model.layers, copy_model.layers):
    new._name=org.name